In [ ]:
import pandas as pd
import requests
from tqdm.notebook import trange, tqdm
import math
import random
import time
import os
import shutil

In [ ]:
imena = {'I':'budetinetakoe', 'N':'anastasiiamerkulova'}
# имя и аккаунт

In [ ]:
def get_obs_to_df(ima, I_ids):
    
    # принимает имена и загружает наблюдения в датасет со столбцами: 'ima','taxon_id', 'obs_id','date','scientific_name', 'english_common_name', 'common_name','uri', 'photos'
    # также принимает список таксонов I_ids, которые нужно загружать с сервера (во избежание загрузки "ненужных" таксонов)
    # возвращает датасет pandas

    page = 1
    pages = 99
    per_page = 200
    url = 'https://api.inaturalist.org/v1/observations?'

    df = pd.DataFrame(columns=['ima','taxon_id', 'obs_id','date','scientific_name', 'english_common_name', 'common_name','uri', 'photos'])
    row_number = 1

    while page <= pages:
        params = {'photos' : 'true',
            'user_login' : ima,
            'quality_grade' : 'research',
            'locale' : 'ru',
            'taxon_id' : I_ids,
            'hrank' : 'species',
            'lrank' : 'form',
            'page' : page,
            'per_page' : per_page}
        response = requests.get(url=url, params=params)
        print(f'Статус: {response.status_code}, на страницу: {per_page}, страница: {page}, всего страниц: {pages}  \n', end='\r')
        json_response = response.json()
        results = json_response['results']
        total_results = json_response['total_results']
        pages = math.ceil(total_results/per_page)

        for i in range(1, len(results)+1):
            obs = results[i-1]
            obs_id = obs['id']
            date = obs['observed_on_details']['date']
            scientific_name = obs['taxon']['name']
            taxon_id = obs['taxon']['id']
            
            try:
                common_name = obs['taxon']['preferred_common_name']
            except:
                common_name = ''
            try:
                english_common_name = json_response['results'][i-1]['taxon']['english_common_name'].capitalize()
            except:
                english_common_name = ''
            uri = obs['uri']

            photos = list()
            for n in range(1, len(json_response['results'][i-1]['photos'])+1):
                photos.append(json_response['results'][i-1]['photos'][n-1]['url'])

            df.loc[row_number] = [ima, taxon_id, obs_id, date, scientific_name, english_common_name, common_name, uri, photos]
            row_number += 1
            
        page += 1
        time.sleep(1)
        
    return df

In [ ]:
def get_obs_unique(df):
    
    # оставляет в датасете только одно случайное наблюдение каждого scientific_name
    # возвращает этот же датасет, с уменьшенным количеством строк
    
    df = df.sample(frac=1).drop_duplicates(subset='taxon_id')
    
    return df

In [ ]:
def get_common_obs(df_I, df_N):
    # берёт два датасета и делает из них один, оставляя только совпадающие taxon_id
    # также добавляет в датасет столбец snamename с первым словом из common_name
    # возвращает объединённый датасет

    df = df_I.merge(right=df_N, how='inner',on='taxon_id')
    df['snamename'] = df.common_name_x.apply(lambda x: x.split(' ')[0])

    return df

In [ ]:
def get_structure(df):
    # берёт общий датасет, идёт по scientific_name и для каждого:
        # создаёт папку с именем scientific_name
        # создаёт в scientific_name подпапки I, N
        # в scientific_name создаёт info.txt
        # в подпапки загружает фотографии 1.jpg, 2.jpg и тд
        # возвращает True если всё ок (это не используется)

    snamename_counts = df.snamename.value_counts()
        
    for s_name in tqdm(df['scientific_name_x']):

        current_row = df[df['scientific_name_x']==s_name]
        parent_dir = '/home/eva/git/iNat/iNat/'
        current_dir = parent_dir + '/' + s_name
        os.mkdir(current_dir)
        I_dir = current_dir+'/I'
        N_dir = current_dir+'/N'
        os.mkdir(I_dir)
        os.mkdir(N_dir)

        with open(current_dir+'/info.txt','w') as fi:
            
            uri_I = current_row['uri_x'].item()
            uri_N = current_row['uri_y'].item()
            scientific_name = current_row['scientific_name_x'].item()
            english_common_name = current_row['english_common_name_x'].item()
            common_name = current_row['common_name_x'].item()
            snamename = current_row['snamename'].item()
            snamename_count = snamename_counts[snamename]

            string_info = f'Научное название: {scientific_name}\n' + \
                f'Общее название: {common_name} \n' + \
                f'Общее название (англ.): {english_common_name}\n' + \
                f'Количество видов {snamename}: {snamename_count}\n' + \
                f'URI I:\n{uri_I}\n' + \
                f'URI N:\n{uri_N}'

            fi.write(string_info)

        for suffix, name_dir in (('_x', I_dir),('_y', N_dir)):
            for i in range(len(current_row['photos'+suffix].item())):

                photo_url = current_row['photos'+suffix].item()[i]
                photo_url = photo_url.replace('square','original')
                photo_name = str(i+1) + '.jpg'
                file_path =  name_dir + '/' + photo_name
                r = requests.get(url=photo_url, stream=True)

                if r.status_code == 200:
                    with open(file_path, 'wb') as f:
                        r.raw.decode_content = True
                        shutil.copyfileobj(r.raw, f)

    return True

In [ ]:
def rename(df):
    # берёт список папок и переименовывает их случайным образом в 001, 002 и тд
    # возвращает True если всё ок (это не используется)
    # закоменченные строки с numbers_strong используются чтобы переименовать уже переименованные папки (из 001 в другое случайное число, например, 098)

    numbers = list(range(1,len(df['scientific_name_x'])+1))
    numbers_strong = numbers.copy()
    random.shuffle(numbers)
    parent_dir = '/home/eva/git/iNat/iNat'

    for s_name, new_name in zip(df['scientific_name_x'], numbers):
    # for strong_number, new_name in zip(numbers_strong, numbers):
        src = parent_dir + '/' + s_name
        # src = parent_dir + '/' + str(strong_number).zfill(3)
        dst = parent_dir + '/' + str(new_name).zfill(3)
        # dst = parent_dir + '/_' + str(new_name).zfill(3)
        os.rename(src, dst)

    print(f'Всего директорий: {len(numbers)} \n')

    return True

In [ ]:
df_I = get_obs_unique(get_obs_to_df(ima=imena['I'], I_ids=''))

I_ids = ','.join(df_I.taxon_id.astype('string'))

print(f'Датасет df_I: {len(df_I)} шт.')

df_N = get_obs_unique(get_obs_to_df(ima=imena['N'], I_ids=I_ids))

print(f'Датасет df_N: {len(df_N)} шт.')

df_I_N = get_common_obs(df_I, df_N)

print(f'Датасет df_I_N: {len(df_I_N)} шт.')

get_structure(df_I_N)

rename(df_I_N)

In [ ]:
# set_I = set(df_I.taxon_id)
# set_I_N = set(df_I_N.taxon_id)
# set_N = set(df_N.taxon_id)
# diff_I_N = set_I-set_I_N
# diff_N_I_N = set_N - set_I_N
# df_I[df_I.taxon_id.isin(diff_I_N)]
# df_N[df_N.taxon_id.isin(diff_N_I_N)]